# Outline

## Training and Tuning ML Models
- Hyperparameter Tuning
    - Algorithms for model tunign
    - Sagemaker automated parameter tuning
    - BERT Automated tuning
    
    - warm start hyperparameter turning
        - speedup the tuning process
    - Check point in machine learning
        - managed spot training; to reduce the cost of training
        
    - Distirbuted training challenges
        - Tune distributed training stragegies
            - data panelism
            - model panelism
            
    - Bring your own container

# Advanced Model Training and Tuning

<img src='./pics/courser outline.png'>

- challenges in training and tuning large machine learning models while optimizing cost

- Popular algorithms for hyperparameter tuning (Automatic)
    - 4 algorithms
    - <u>Grid search</u>
        - specify parameters to tune
        - specify the range of the values for each parameter
        - the algorithm will train the model of every combination and find the best parameter

        - Pros
            - Explore all possible combintions
            - works for small number of parameters
        - Cons
            - time consuming for large number of parameters
            - doesn't scale for large number of parameters
        
    - <u>Random search</u>
        - Specify the hyperparmter to tune
        - Define the search space and stop criteria
            - some stop criteria may be,
                - time elapsed
                - maximum number of models to be trained
                
        - Test random combinations within the search space
        - select the best combination
        
        - Pros
            - Faster than the grid search
        - Cons
            - May miss the best performing hyperparameters
        
    - <u>Bayesian optimization</u>
        - Hyperparameter is treated as a regression problem
        - Start with random hyperprameters
        - Narrow down the search space around a better performing hyperprameter
        
        - Pros
            - more efficient in finding the best hyperparameter
            
        - Cons
            - requires sequential execution
            - migh get stuck in local minima
            
            - <img src='./pics/local minima.png'>
        
            - this is a prominent probelm when we are using gradient discent for opimizing the probelm
                
    - <u>Hyperband</u>
        - <b>Bandit-based</b> approach (exploitations, explorations)
        - Start from random hyperparameters
        - Explore set of hyperparameters for few iterations (after exploration abandon half of the least performing sets)
        - choose best and explore longer
        - Repeat until max_iterations reached or one candidate left
        
        - Pros
            - Spends time efficiently
        - Cons
            - might discard good candidates early that converge slowly

# Tune a BERT-based Text Classifer

# <u>ML model training challenges</u>

# Checkpointing

# Distributed Training Strategies

# Custom Algorithms with Amazon SageMaker